In [34]:
import os

In [35]:
%pwd

'f:\\Self Learning\\car_price_prediction'

In [5]:
os.chdir("../")

In [6]:
%pwd

'f:\\Self Learning\\car_price_prediction'

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
@dataclass
class PrepareBaseModelConfig:
    base_model_path: Path
    updated_base_model_path: Path
    feature_columns: List[str]
    target_column: str
    test_data_path: Path

@dataclass
class PreprocessingConfig:
    target_column: str
    test_size: float
    random_state: int

In [37]:
from car_price_prediction.constants import *
from car_price_prediction.utils.common import read_yaml, create_directories

In [49]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_preprocessing_config(self) -> PreprocessingConfig:
        preprocessing = self.config.preprocessing
        params = self.params

        preprocessing_config = PreprocessingConfig(
            target_column=preprocessing.target_column,
            test_size=params.TEST_SIZE,
            random_state=params.RANDOM_STATE
        )

        return preprocessing_config
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        model_params = self.params['model']
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            feature_columns=model_params["feature_columns"],
            target_column=model_params["target_column"],
            updated_base_model_path=Path(config.updated_base_model_path),
            params_learning_rate=self.params.LEARNING_RATE
        )

        return prepare_base_model_config


In [50]:
import os
import urllib.request as request
from zipfile import ZipFile

In [51]:
import os
import joblib
from sklearn.linear_model import LinearRegression
from pathlib import Path
from dataclasses import dataclass
from typing import Optional
import pandas as pd

from car_price_prediction.entity.config_entity import PrepareBaseModelConfig

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.model = None
        self.full_model = None

    def get_base_model(self):
        """Initialize base Linear Regression model"""
        self.model = LinearRegression()
        self.save_model(self.config.base_model_path, self.model)

    @staticmethod
    def _prepare_full_model(model: LinearRegression, X: pd.DataFrame, y: pd.Series):
        """Train the linear regression model on the dataset"""
        model.fit(X, y)
        return model

    def update_base_model(self):
        """Load data and train model"""
        df = pd.read_csv(self.config.test_data_path)

        X = df[self.config.feature_columns]
        y = df[self.config.target_column]

        self.full_model = self._prepare_full_model(
            model=self.model,
            X=X,
            y=y
        )
        self.save_model(self.config.updated_base_model_path, self.full_model)

    @staticmethod
    def save_model(path: Path, model: LinearRegression):
        """Save model using joblib"""
        joblib.dump(model, path)


In [52]:
from car_price_prediction.entity.config_entity import (
    DataIngestionConfig,
    PreprocessingConfig,
    PrepareBaseModelConfig
)
from car_price_prediction.utils.common import read_yaml
from car_price_prediction.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        return DataIngestionConfig(
            local_data_file=Path(self.config["data_ingestion"]["local_data_file"])
        )

    def get_preprocessing_config(self) -> PreprocessingConfig:
        return PreprocessingConfig(
            target_column=self.params["preprocessing"]["target_column"]
        )

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        model_config = self.config["prepare_base_model"]
        model_params = self.params["model"]
        return PrepareBaseModelConfig(
            base_model_path=Path(model_config["base_model_path"]),
            updated_base_model_path=Path(model_config["updated_base_model_path"]),
            feature_columns=model_params["feature_columns"],
            target_column=model_params["target_column"],
            test_data_path=Path(model_config["test_data_path"])
        )


In [54]:
from car_price_prediction.config.configuration import ConfigurationManager
from car_price_prediction.components.prepare_base_model import PrepareBaseModel

config = ConfigurationManager()
prepare_base_model_config = config.get_prepare_base_model_config()

prepare_model = PrepareBaseModel(config=prepare_base_model_config)
prepare_model.get_base_model()
prepare_model.update_base_model()


ImportError: cannot import name 'PrepareBaseModel' from 'car_price_prediction.components.prepare_base_model' (F:\Self Learning\car_price_prediction\src\car_price_prediction\components\prepare_base_model.py)